### Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
import nltk
nltk.download('stopwords')
# stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

import warnings
warnings.simplefilter('ignore')

import nltk
stop = set(nltk.corpus.stopwords.words('english'))
print(stop)

In [135]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Get Data

In [6]:
tweet= pd.read_csv('./Data/train.csv')
test=pd.read_csv('./Data/test.csv')
submission = pd.read_csv("./Data/sample_submission.csv")

In [7]:
# Correct wrong data
# From https://www.kaggle.com/wrrosa/keras-bert-using-tfhub-modified-train-data - 
# author of this kernel read tweets in training data and figure out that some of them have errors:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
tweet.loc[tweet['id'].isin(ids_with_target_error),'target'] = 0
tweet[tweet['id'].isin(ids_with_target_error)]

,id,keyword,location,text,target
229,328,annihilated,NaN,Ready to get annihilated for the BUCS game,0
301,443,apocalypse,NaN,Short Reading\n\nApocalypse 21:1023 \n\nIn the...,0
356,513,army,Studio,But if you build an army of 100 dogs and their...,0
1822,2619,crashed,NaN,My iPod crashed..... \n#WeLoveYouLouis \n#MTVH...,0
2536,3640,desolation,"Quilmes , Arg",This desperation dislocation\nSeparation conde...,0
2715,3900,devastated,PG Chillin!,Man Currensy really be talkin that talk... I'd...,0
3024,4342,dust%20storm,chicago,Going to a fest? Bring swimming goggles for th...,0
4068,5781,forest%20fires,NaN,Campsite recommendations \nToilets /shower \nP...,0
4609,6552,injury,Saint Paul,My prediction for the Vikings game this Sunday...,0
4611,6554,injury,NaN,Dante Exum's knee injury could stem Jazz's hop...,0


In [8]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

There are 7613 rows and 5 columns in train
There are 3263 rows and 4 columns in test


In [9]:
tweet.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [196]:
df=pd.concat([tweet,test])
tweet.shape
test.shape
df.shape

(7613, 5)

(3263, 4)

(10876, 5)

## Useful Preprocessing/Data cleaning

#### Removing URLs

In [82]:
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"

In [83]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

remove_URL(example)

'New competition launched :'

In [84]:
df['text']=df['text'].apply(lambda x : remove_URL(x))

#### Removing HTML tags

In [85]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

In [86]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
print(remove_html(example))


Real or Fake
Kaggle 
getting started



In [87]:
df['text']=df['text'].apply(lambda x : remove_html(x))

#### Not removing but handling emoji and emoticons

In [88]:
# installation and importing the library
!pip install emot
import emot

In [89]:
def replace_emojis(text):
    emoji_details=emot.emoji(text)
    meaning=re.compile(r':')
    emojis_meaning=""
    for m in emoji_details['mean']:
        emojis_meaning=emojis_meaning+" "+meaning.sub(r'',m)
    return emojis_meaning

In [90]:
def replace_emoticons(text):
    emoticon_details=emot.emoticons(text)
    emoticon_meaning=""
    print(emoticon_details['mean'])
    for m in emoticon_details['mean']:
        emoticon_meaning=emoticon_meaning+" "+ str(m)
    return emoticon_meaning

In [97]:
example_df= pd.DataFrame(["The weather is ☁️ :), we might need to carry our ☂️ :("])
example_df.columns=['text']
example_df['emojis_meaning']=example_df['text'].apply(lambda x: replace_emojis(x))
example_df['emoticon_meaning']=example_df['text'].apply(lambda x: replace_emoticons(x))
print(example_df)

['Happy face or smiley', 'Frown, sad, andry or pouting']
                                                text   emojis_meaning  \
0  The weather is ☁️ :), we might need to carry o...   cloud umbrella   

                                    emoticon_meaning  
0   Happy face or smiley Frown, sad, andry or pou...  


In [92]:
df['emojis_meaning']=df['text'].apply(lambda x: replace_emojis(x))
# df['emoticon_meaning']=df['text'].apply(lambda y: replace_emoticons(y))

In [93]:
print(df[0:5])

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target emojis_meaning  
0     1.0                 
1     1.0                 
2     1.0                 
3     1.0                 
4     1.0                 


In [94]:
df['emojis_meaning'].describe(include="all")

count     10876
unique        2
top            
freq      10862
Name: emojis_meaning, dtype: object

In [98]:
df['emojis_meaning'].unique

<bound method Series.unique of 0        
1        
2        
3        
4        
       ..
3258     
3259     
3260     
3261     
3262     
Name: emojis_meaning, Length: 10876, dtype: object>

#### Removing emoji and emoticons, in case handling is not possible

In [101]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [102]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

#### Removing punctuations

In [103]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))

I am a king


In [104]:
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [122]:
df.describe()

,id,target
count,10876.000000,7613.000000
mean,5437.500000,0.427558
std,3139.775098,0.494757
min,0.000000,0.000000
25%,2718.750000,0.000000
50%,5437.500000,0.000000
75%,8156.250000,1.000000
max,10875.000000,1.000000


In [126]:
df

,id,keyword,location,text,target,emojis_meaning
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1.0,
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1.0,
2,5,NaN,NaN,All residents asked to shelter in place are be...,1.0,
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1.0,
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1.0,
...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN,
3259,10865,NaN,NaN,Storm in RI worse than last hurricane My citya...,NaN,
3260,10868,NaN,NaN,Green Line derailment in Chicago,NaN,
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook HWO,NaN,


In [141]:
training_df=df[df['target'].notnull()]

In [144]:
oot_df=df[df['target'].isnull()]

In [146]:
training_df.shape
oot_df.shape

(7613, 6)

(3263, 6)

In [158]:
oot_df.describe(include="all")

,id,keyword,location,text,target,emojis_meaning
count,3263.000000,3237,2158,3263,0.0,3263
unique,NaN,221,1602,3107,NaN,2
top,NaN,deluged,New York,US National Park Services Tonto National Fores...,NaN,
freq,NaN,23,38,11,NaN,3259
mean,5427.152927,NaN,NaN,NaN,NaN,NaN
std,3146.427221,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN
25%,2683.000000,NaN,NaN,NaN,NaN,NaN
50%,5500.000000,NaN,NaN,NaN,NaN,NaN
75%,8176.000000,NaN,NaN,NaN,NaN,NaN


In [159]:
oot_df = oot_df.drop('target', 1)

### Trying simple model now

In [183]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = training_df["text"].tolist()
list_labels = training_df["target"].tolist()
oot_corpus = oot_df["text"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=32)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)
oot_df_counts = count_vectorizer.transform(oot_corpus)

In [184]:
len(X_train)
# len(X_train_counts)

6090

In [185]:

## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [186]:
scores = model_selection.cross_val_score(clf, X_train_counts, y_train, cv=3, scoring="f1")
scores

array([0.71800948, 0.71281117, 0.73144876])

In [187]:
scores = model_selection.cross_val_score(clf, X_test_counts, y_test, cv=3, scoring="f1")
scores

array([0.6122449 , 0.63829787, 0.6870229 ])

In [171]:
clf.fit(X_train_counts, y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [172]:
sample_submission = pd.read_csv("./Data/sample_submission.csv")

In [173]:
sample_submission["target"] = clf.predict(oot_df_counts)

In [175]:
sample_submission.describe()

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.380325
std,3146.427221,0.485541
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [ ]:
#why am i getting score of 0?

In [176]:
sample_submission.to_csv("submission.csv", index=False)

### Trying simple model on whole dataset

In [188]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = training_df["text"].tolist()
list_labels = training_df["target"].tolist()
oot_corpus = oot_df["text"].tolist()

# X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
#                                                                                 random_state=32)

X_train_counts, count_vectorizer = cv(list_corpus)
# X_test_counts = count_vectorizer.transform(X_test)
oot_df_counts = count_vectorizer.transform(oot_corpus)

In [189]:
clf = linear_model.RidgeClassifier()

In [190]:
clf.fit(X_train_counts, list_labels)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [192]:
scores = model_selection.cross_val_score(clf, X_train_counts, list_labels , cv=3, scoring="f1")
scores

array([0.6190226 , 0.55024876, 0.62953995])

In [193]:
sample_submission = pd.read_csv("./Data/sample_submission.csv")
sample_submission["target"] = clf.predict(oot_df_counts)
sample_submission.describe()
sample_submission.to_csv("submission.csv", index=False)

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.375421
std,3146.427221,0.484306
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


### TF-IDF vectorizer

In [197]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(list_corpus)
X_oot_tfidf = tfidf_vectorizer.transform(oot_corpus)


In [201]:
clf = linear_model.RidgeClassifier()

In [204]:
print(len(list_labels))

7613


In [205]:
clf.fit(X_train_tfidf, list_labels)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [206]:
scores = model_selection.cross_val_score(clf, X_train_tfidf, list_labels , cv=3, scoring="f1")
scores

array([0.63322185, 0.60010565, 0.68452983])

In [207]:
sample_submission = pd.read_csv("./Data/sample_submission.csv")
sample_submission["target"] = clf.predict(X_oot_tfidf)
sample_submission.describe()
sample_submission.to_csv("submission.csv", index=False)

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.368986
std,3146.427221,0.482604
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000
